In [1]:
import json
import numpy as np
import os
import pandas as pd
import simplejson
import string
from copy import deepcopy
from pprint import pprint
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import re

DATA_DIR = '{}/research/data'.format(os.getenv('HOME'))
seed = 42
shuffler = random.Random(seed)

num_distractor_paragraphs = 1

In [5]:
# Drop passages only on train
with open('{}/hotpot-squad/hotpot_train_v1.json'.format(DATA_DIR)) as f:
    data = json.loads(f)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ethanperez/research/data/hotpot-squad/hotpot_train_v1.json'

In [3]:
data_new = deepcopy(data)
for example_no in tqdm(range(len(data_new['data']))):
    if '_id' not in data_new['data'][example_no]['paragraphs'][0]:
        continue # Don't modify SQuAD questions
    gold_paragraphs = []
    distractor_paragraphs = []
    for paragraph in data_new['data'][example_no]['paragraphs']:
        if len(paragraph['qas'][0]['answers']) > 0:
            gold_paragraphs.append(paragraph)
        else:
            distractor_paragraphs.append(paragraph)
    shuffler.shuffle(distractor_paragraphs)
    data_new['data'][example_no]['paragraphs'] = gold_paragraphs + distractor_paragraphs[:num_distractor_paragraphs]

100%|██████████| 90889/90889 [00:01<00:00, 80148.47it/s]


In [4]:
save_dir = '{}/hotpot-squad-distractors={}'.format(DATA_DIR, num_distractor_paragraphs)
os.makedirs(save_dir, exist_ok=True)
with open('{}/hotpot_train_v1.json'.format(save_dir), 'w') as f:
    json.dump(data_new, f, indent=2)
with open('{}/hotpot-squad/hotpot_train_v1.json'.format(DATA_DIR), 'r') as f:
    data_dev = json.load(f)
with open('{}/hotpot_dev_distractor_v1.json'.format(save_dir), 'w') as f:
    json.dump(data_dev, f, indent=2)